### 01. High Price Predict Model with Linear Regression

In [1]:
# 시간 측정 시작
import time
start = time.time()

# 라이브러리
import pandas as pd
kospi = pd.read_csv('../data/KOSPI.csv')

# KOSPI에 상장된 종목 코드
kospi['종목코드'] = kospi['종목코드'].apply(lambda x : str(x).rjust(6, '0'))
code_list = kospi['종목코드'].unique()

# 라이브러리
import numpy as np

# 진행과정 Check
check = 0

# 결과 저장 데이터프레임
result_df = pd.DataFrame()

# 코드 별 진행
for code in code_list[:] : 

    # Hyper Parameter
    tr_name = 'TR_SCHART'
    term = 'D'
    start_date = '20180510'   # 학습 기간 시작점
    end_date = '20200229'     # 학습 기간 종료점
    Lookup = '9999' 
    
    # url
    url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=1' +\
          '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup

    # url open
    from urllib.request import urlopen
    url_page = urlopen(url)

    # json 파일로 받아오기
    import json
    url_data = json.loads(url_page.read())

    # json을 데이터프레임으로 바꾸기
    import pandas as pd
    data = pd.DataFrame(url_data)
    
    # Data Column명 변경
    data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr',
                    'rock','volume','volume_price']
    data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    
    # 순서 뒤집기
    data = data[::-1]
    data = data.set_index('date')
    
    # 필요한 변수만 추출
    data = data[['open', 'high', 'low', 'close', 'volume', 'volume_price']]

    # 만약에 데이터가 비어있다면 그 종목은 PASS
    if data["open"][0] == '' : 
        continue
    
    # 변수 속성 정수로 바꾸기
    for column in data.columns : 
        data[column] = data[column].apply(lambda x : int(x))
        
    # 거래량과 거래가격을 모두 로그 변환
    data['volume'] = data['volume'].apply(lambda x : np.log(x+1))
    data['volume_price'] = data['volume_price'].apply(lambda x : np.log(x+1))
    
    # 각각의 피쳐들에 대해 
    for column in ['open', 'high', 'low', 'close', 'volume', 'volume_price'] : 
        for i in range(1,5,1) : 
            data['{}_shift_{}'.format(column,i)] = data[column].shift(i)
    
    # 타겟 생성 : 다음날의 고가를 예측
    data['y'] = data['high'].shift(-1)
    
    # NaN이 들어가 있는 행 제거
    data = data.dropna()
    
    # Input과 Target을 분리
    X = data[data.columns[:-1]]
    y = data['y']
    
    # Linear Regression
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(X, y)
    
    # Test Data 생성하기
    start_date, end_date = '20200301', '20200320'
    url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=1' +\
          '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup
    url_page = urlopen(url)
    url_data = json.loads(url_page.read())
    data = pd.DataFrame(url_data)
    data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr',
                    'rock','volume','volume_price']
    data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    data = data[::-1]
    data = data.set_index('date')
    data = data[['open', 'high', 'low', 'close', 'volume', 'volume_price']]
    for column in data.columns : 
        data[column] = data[column].apply(lambda x : int(x))
    data['volume'] = data['volume'].apply(lambda x : np.log(x+1))
    data['volume_price'] = data['volume_price'].apply(lambda x : np.log(x+1))
    for column in ['open', 'high', 'low', 'close', 'volume', 'volume_price'] : 
        for i in range(1,5,1) : 
            data['{}_shift_{}'.format(column,i)] = data[column].shift(i)
    data['y'] = data['high'].shift(-1)
        
    data = data[-1:]
    
    data = data[data.columns[:-1]]

    # 필요한것들
    result = dict({'종목' : code,
                   '어제 날짜' : data.index.values[0],
                   '어제 종가' : int(data['close'].values),
                   '오늘 고가' : int(round(reg.predict(data)[0])),
                   '오늘 고가 변동폭 예측' : 100*(int(round(reg.predict(data)[0]))-int(data['close'].values))/\
                                     int(data['close'].values)})
    result_df = result_df.append(result, ignore_index=True)
    
    # 각 종목별 결과 출력
    if check % 100 == 0 : 
        print('{:4} / {:4} 종목 ====> {:7.2f}% 완료  {:.4f}'.format(check+1,2144,100*check/2144,time.time()-start))
    check += 1

# 전체 완료
print('{:4} / {:4} 종목 ====> {:7.2f}% 완료  {:.4f}'.format(check + 1,2144,100*check/2144,time.time()-start))

# 시간측정 완료
print("\n\nall time :", time.time() - start)

   1 / 2144 종목 ====>    0.00% 완료  1.2200
 101 / 2144 종목 ====>    4.66% 완료  51.6224
 201 / 2144 종목 ====>    9.33% 완료  102.1294
 301 / 2144 종목 ====>   13.99% 완료  367.9102
 401 / 2144 종목 ====>   18.66% 완료  418.7151
 501 / 2144 종목 ====>   23.32% 완료  469.8113
 601 / 2144 종목 ====>   27.99% 완료  521.8700
 701 / 2144 종목 ====>   32.65% 완료  574.3417
 801 / 2144 종목 ====>   37.31% 완료  626.2585
 901 / 2144 종목 ====>   41.98% 완료  677.8531
1001 / 2144 종목 ====>   46.64% 완료  729.1071
1101 / 2144 종목 ====>   51.31% 완료  781.8289
1201 / 2144 종목 ====>   55.97% 완료  836.4658
1301 / 2144 종목 ====>   60.63% 완료  888.0588
1401 / 2144 종목 ====>   65.30% 완료  939.3519
1501 / 2144 종목 ====>   69.96% 완료  990.5911
1601 / 2144 종목 ====>   74.63% 완료  1043.6507
1701 / 2144 종목 ====>   79.29% 완료  1094.8723
1801 / 2144 종목 ====>   83.96% 완료  1147.9837
1901 / 2144 종목 ====>   88.62% 완료  1199.9898
2001 / 2144 종목 ====>   93.28% 완료  1250.6461
2101 / 2144 종목 ====>   97.95% 완료  1302.6210
2145 / 2144 종목 ====>  100.00% 완료  1324.7947


all t

### 02. 예측 결과

In [2]:
result_df = result_df[['어제 날짜', '종목', '어제 종가', '오늘 고가', '오늘 고가 변동폭 예측']]

In [3]:
result_df['오늘 고가 변동폭 예측'] = result_df['오늘 고가 변동폭 예측'].apply(lambda x : round(x,2))

In [4]:
result_df

,어제 날짜,종목,어제 종가,오늘 고가,오늘 고가 변동폭 예측
0,20200320,060310,1510.0,1649.0,9.21
1,20200320,095570,2040.0,1645.0,-19.36
2,20200320,068400,5450.0,6081.0,11.58
3,20200320,006840,15600.0,14402.0,-7.68
4,20200320,054620,5870.0,6178.0,5.25
...,...,...,...,...,...
2139,20200320,189980,869.0,933.0,7.36
2140,20200320,000540,1525.0,1774.0,16.33
2141,20200320,003280,244.0,272.0,11.48
2142,20200320,037440,3435.0,3551.0,3.38


### 03. 내일 고가 변동폭이 가장 높은 TOP 10

In [5]:
result_df = result_df.sort_values(by='오늘 고가 변동폭 예측', ascending=False).set_index('종목')

In [6]:
result_df = result_df[result_df['오늘 고가 변동폭 예측'] < 30.00]

In [7]:
result_df[:10]

,어제 날짜,어제 종가,오늘 고가,오늘 고가 변동폭 예측
종목,,,,
141020,20200320,108.0,140.0,29.63
054940,20200320,690.0,894.0,29.57
057030,20200320,3875.0,5000.0,29.03
033340,20200320,2035.0,2625.0,28.99
033320,20200320,2155.0,2775.0,28.77
072950,20200320,3050.0,3918.0,28.46
058530,20200320,222.0,284.0,27.93
196490,20200320,698.0,892.0,27.79
150840,20200320,2295.0,2924.0,27.41


### 04. 내일 고가 변동폭이 가장 높은 TOP 11 ~ 20

In [8]:
result_df[10:20]

,어제 날짜,어제 종가,오늘 고가,오늘 고가 변동폭 예측
종목,,,,
289010,20200320,6530.0,8200.0,25.57
008700,20200320,1080.0,1354.0,25.37
002990,20200320,4505.0,5613.0,24.59
035620,20200320,1415.0,1759.0,24.31
015260,20200320,183.0,227.0,24.04
203650,20200320,1280.0,1586.0,23.91
087010,20200320,12500.0,15467.0,23.74
065620,20200320,3250.0,4021.0,23.72
036480,20200320,8660.0,10698.0,23.53


### 05. 내일 고가 변동폭이 가장 낮은 BOTTOM 10

In [9]:
result_df = result_df[result_df['오늘 고가 변동폭 예측'] > -30.00]
result_df[-10:]

,어제 날짜,어제 종가,오늘 고가,오늘 고가 변동폭 예측
종목,,,,
186230,20200320,5700.0,4314.0,-24.32
001380,20200320,694.0,525.0,-24.35
122350,20200320,1160.0,870.0,-25.00
175140,20200320,3095.0,2321.0,-25.01
011160,20200320,1240.0,924.0,-25.48
086250,20200320,1080.0,800.0,-25.93
279600,20200320,2770.0,2045.0,-26.17
222420,20200320,863.0,634.0,-26.54
196700,20200320,964.0,695.0,-27.90


### 06. 특정 종목 검색

In [10]:
# result_df.loc['019770']